# Client Recommendation

* Frequent Pattern mining doesn't work well for this data set.
* Therefore I'm using similarity recommendation.
  * The assumption here is, a client will purchase similar products they had purchased before.

In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
all_order_train = pd.read_pickle('../all_order_train.pkl')
all_order_test = pd.read_pickle('../all_order_test.pkl')
print(all_order_train.shape, all_order_test.shape)

all_order_train.head()

(33720820, 12) (98286, 12)


order_id  user_id  order_number  order_hour_of_day  product_id  \
0   2539329        1             1                  8         196   
1   2231262       31            17                 11         196   
2   3058369      195            34                 10         196   
3   2257155      951             3                 20         196   
4   1121647      992             7                 10         196   

  purchase_date                              merchant product_name  price  \
0    2019-02-13  L&M Markets (Hometown Grocers Co-op)         Soda   3.08   
1    2019-03-10  L&M Markets (Hometown Grocers Co-op)         Soda   3.08   
2    2019-02-25  L&M Markets (Hometown Grocers Co-op)         Soda   3.08   
3    2019-03-09  L&M Markets (Hometown Grocers Co-op)         Soda   3.08   
4    2019-03-08  L&M Markets (Hometown Grocers Co-op)         Soda   3.08   

         aisle department  week_number  
0  soft drinks  beverages            7  
1  soft drinks  beverages           10  
2  soft drinks  beverages            9  
3  soft drinks  beverages           10  
4  soft drinks  beverages           10

In [8]:
shared_clients = set(all_order_train['user_id'].values).intersection(all_order_test['user_id'].values)
print(len(shared_clients))

3457


In [10]:
train_shared_client_week_df = all_order_train.loc[all_order_train['user_id']\
                                                  .isin(list(shared_clients))][['user_id', 'week_number']]\
                                                  .astype('str').drop_duplicates()\
                                                  .groupby(['user_id'], as_index=False)['week_number']\
                                                  .agg(['count']).reset_index()\
                                                  .sort_values(['count'], ascending=False)

train_shared_client_week_df.head()

user_id  count
1549  183503      5
3210   86398      5
290   116891      5
3196   85846      5
1494  180563      5

In [11]:
target_client = 183503

## Client Match Score & Discount

In [27]:
merchant_sales_df = all_order_train[['merchant', 'user_id', 'price']].drop_duplicates()\
          .groupby(['merchant'], as_index=False)\
          .agg({'user_id': 'count', 'price': 'sum'})

merchant_sales_df.head()

merchant  user_id         price
0       49th Parallel Grocery  1211766  1.452954e+07
1           Asian Food Centre  1210187  1.451086e+07
2               Askew's Foods  1203583  1.443146e+07
3  Avril (Health Supermarket)  1200036  1.439485e+07
4               Calgary Co-op  1017673  1.218554e+07

In [28]:
merchant_sales_df['avg_sales_per_client'] = round(merchant_sales_df['price']/merchant_sales_df['user_id'], 4)

merchant_sales_df.head()

merchant  user_id         price  avg_sales_per_client
0       49th Parallel Grocery  1211766  1.452954e+07               11.9904
1           Asian Food Centre  1210187  1.451086e+07               11.9906
2               Askew's Foods  1203583  1.443146e+07               11.9904
3  Avril (Health Supermarket)  1200036  1.439485e+07               11.9953
4               Calgary Co-op  1017673  1.218554e+07               11.9739

In [29]:
client_purchase_df = all_order_train[['user_id', 'price', 'order_id']].drop_duplicates()\
          .groupby(['user_id'], as_index=False)\
          .agg({'price': 'sum', 'order_id': 'count'})

client_purchase_df.head()

user_id    price  order_id
0        1   560.43        70
1        2  2685.71       226
2        3   994.48        88
3        4   282.01        18
4        5   543.37        46

In [30]:
client_purchase_df['avg_purchase'] = round(client_purchase_df['price']/client_purchase_df['order_id'], 4)

client_purchase_df.head()

user_id    price  order_id  avg_purchase
0        1   560.43        70        8.0061
1        2  2685.71       226       11.8837
2        3   994.48        88       11.3009
3        4   282.01        18       15.6672
4        5   543.37        46       11.8124

In [34]:
merchant_sales_df['merchant_match_score'] = abs(1 - 
                            client_purchase_df.loc[client_purchase_df['user_id']==target_client]['avg_purchase'].values[0]\
                                                /merchant_sales_df['avg_sales_per_client'])
merchant_sales_df['discount'] = merchant_sales_df['price'] * 0.0000007

merchant_sales_df = merchant_sales_df.sort_values(by='merchant_match_score')
merchant_sales_df.head()

merchant  user_id       price  avg_sales_per_client  \
44        Sunterra Market    87915  1042779.22               11.8612   
45      TaiKo Supermarket    79776   948310.74               11.8872   
37         Planet Organic   155315  1848801.80               11.9036   
36  Panchvati Supermarket   189487  2257196.69               11.9121   
25               Lococo's   352710  4205428.28               11.9232   

    merchant_match_score  discount  
44              0.158803  0.729945  
45              0.160643  0.663818  
37              0.161800  1.294161  
36              0.162398  1.580038  
25              0.163178  2.943800

## Product Similarity Recommendation

In [36]:
all_order_train.head()

order_id  user_id  order_number  order_hour_of_day  product_id  \
0   2539329        1             1                  8         196   
1   2231262       31            17                 11         196   
2   3058369      195            34                 10         196   
3   2257155      951             3                 20         196   
4   1121647      992             7                 10         196   

  purchase_date                              merchant product_name  price  \
0    2019-02-13  L&M Markets (Hometown Grocers Co-op)         Soda   3.08   
1    2019-03-10  L&M Markets (Hometown Grocers Co-op)         Soda   3.08   
2    2019-02-25  L&M Markets (Hometown Grocers Co-op)         Soda   3.08   
3    2019-03-09  L&M Markets (Hometown Grocers Co-op)         Soda   3.08   
4    2019-03-08  L&M Markets (Hometown Grocers Co-op)         Soda   3.08   

         aisle department  week_number  
0  soft drinks  beverages            7  
1  soft drinks  beverages           10  
2  soft drinks  beverages            9  
3  soft drinks  beverages           10  
4  soft drinks  beverages           10

In [45]:
# Client most purchased products

client_products_df = all_order_train.loc[all_order_train['user_id']==target_client][['order_id', 'product_id',
                                                                                      'product_name', 'price', 'week_number']]

client_products_df.head()

order_id  product_id       product_name  price  week_number
56117     1874492         196               Soda   3.08           18
2249475     77864       21386         Smartwater   7.48           18
2250048   3287250       21386         Smartwater   7.48           18
2251343   1874492       21386         Smartwater   7.48           18
3825283    333175       46149  Zero Calorie Cola   3.15           20

In [49]:
client_purchase_amount_df = client_products_df[['product_name', 'order_id']].drop_duplicates()\
          .groupby(['product_name'], as_index=False)['order_id']\
          .agg(['count']).reset_index()\
          .sort_values(['count'], ascending=False)

client_purchase_amount_df.head()

product_name  count
6          Hearts Of Palm      8
12             Smartwater      8
0          Baby Cucumbers      6
15      Zero Calorie Cola      4
2   Cinnamon Toast Crunch      3

In [50]:
favorite_prod_lst = client_purchase_amount_df.loc[client_purchase_amount_df['count'] >= 6]['product_name'].values
favorite_prod_lst

array(['Hearts Of Palm', 'Smartwater', 'Baby Cucumbers'], dtype=object)

In [51]:
merchant_prod_df = pd.read_pickle('../merchant_product.pkl')
print(merchant_prod_df.shape)

prod_category = merchant_prod_df[['aisle_id', 'department_id', 'aisle', 'department']].drop_duplicates()
prod_category = prod_category.sort_values(by=['aisle_id', 'department_id'])
print(prod_category.shape)

prod_category = prod_category.sort_values(by=['department_id', 'aisle_id'])
print(prod_category.shape)

prod_df = merchant_prod_df[['product_id', 'product_name', 'aisle_id', 'department_id', 'aisle', 'department']].drop_duplicates()
print(prod_df.shape)

prod_df['norm_aisle'] = round((prod_df['aisle_id'] - min(prod_df['aisle_id']))/(max(prod_df['aisle_id']) - min(prod_df['aisle_id'])), 4)
prod_df['norm_dept'] = round((prod_df['department_id'] - min(prod_df['department_id']))/(max(prod_df['department_id']) - min(prod_df['department_id'])), 4)

print(min(prod_df['norm_aisle']), max(prod_df['norm_aisle']), min(prod_df['norm_dept']), max(prod_df['norm_dept']))

prod_df.head()

(1439026, 8)
(134, 4)
(134, 4)
(49685, 6)
0.0 1.0 0.0 1.0


product_id                                       product_name  aisle_id  \
0           196                                               Soda        77   
49        46149                                  Zero Calorie Cola        77   
98        17769  Diet Caffeine Free Naturally Flavored Lemon Li...        77   
147       21108                          Cherry Vanilla Creme Soda        77   
196       39409  Ginger Root Beer, Naturally Flavored Zero Calo...        77   

     department_id        aisle department  norm_aisle  norm_dept  
0                7  soft drinks  beverages      0.5714        0.3  
49               7  soft drinks  beverages      0.5714        0.3  
98               7  soft drinks  beverages      0.5714        0.3  
147              7  soft drinks  beverages      0.5714        0.3  
196              7  soft drinks  beverages      0.5714        0.3

In [52]:
def get_prod_similarity(target_aisle, target_dept, aisle, dept):
    similarity = 1 - math.sqrt((pow(target_aisle - aisle, 2) + (pow(target_dept - dept, 2)))/2)
    
    return round(similarity, 4)

In [74]:
import warnings
warnings.filterwarnings("ignore")

prod_lst = []

for target_prod in favorite_prod_lst:
    other_prod_df = prod_df.loc[prod_df['product_name'] != target_prod]
    
    target_aisle = prod_df.loc[prod_df['product_name'] == target_prod]['norm_aisle'].values[0]
    target_dept = prod_df.loc[prod_df['product_name'] == target_prod]['norm_dept'].values[0]

    other_prod_df['similarity'] = other_prod_df.apply(lambda r: get_prod_similarity(target_aisle, target_dept,
                                                                                    r['norm_aisle'], r['norm_dept']), axis=1)
    other_prod_df = other_prod_df.sort_values(by='similarity', ascending=False)
    print(target_prod)
    print('Recommended products: ', list(other_prod_df['product_name'].values[0:4]))
    prod_lst.extend(list(other_prod_df['product_name'].values[0:4]))
    print()
    
prod_lst

Hearts Of Palm
Recommended products:  ['Glitter Candles', 'Organic Tomato Paste', '100% Organic Diced Tomatoes', 'Organic Strained Tomatoes']

Smartwater
Recommended products:  ['Dry Cola Soda, Lightly Sweetened', 'Elderflower Tonic Water', 'Lemon Cucumber Sparkling Water', 'Cantaloupe Grapefruit Sparkling Water']

Baby Cucumbers
Recommended products:  ['Mini Seedless Watermelon Pack', 'Green Seedless Grapes', 'Red Bell Peppers', 'Fresh Cut Butternut Squash']



['Glitter Candles',
 'Organic Tomato Paste',
 '100% Organic Diced Tomatoes',
 'Organic Strained Tomatoes',
 'Dry Cola Soda, Lightly Sweetened',
 'Elderflower Tonic Water',
 'Lemon Cucumber Sparkling Water',
 'Cantaloupe Grapefruit Sparkling Water',
 'Mini Seedless Watermelon Pack',
 'Green Seedless Grapes',
 'Red Bell Peppers',
 'Fresh Cut Butternut Squash']

* I can't believe "glitter Candles" is in "canned jarred vegetables" aisle 😱😱... Feel like this dataset was cleaned by computer vision.
* But think again, lightening a candle while eating salad is, romantic and cozy 💖💖
* And overall, this type of product recommendation does look interesting and useful ❣❣

## Recommend Merchants

* There is no geo location data here, otherwise will definitely consider the geo part.
* Rank top matched merchants with total amount of money the client can save.

In [61]:
merchant_sales_df.head()

merchant  user_id       price  avg_sales_per_client  \
44        Sunterra Market    87915  1042779.22               11.8612   
45      TaiKo Supermarket    79776   948310.74               11.8872   
37         Planet Organic   155315  1848801.80               11.9036   
36  Panchvati Supermarket   189487  2257196.69               11.9121   
25               Lococo's   352710  4205428.28               11.9232   

    merchant_match_score  discount  
44              0.158803  0.729945  
45              0.160643  0.663818  
37              0.161800  1.294161  
36              0.162398  1.580038  
25              0.163178  2.943800

In [64]:
top_matched_merchant_lst = merchant_sales_df['merchant'].values[0:7]
top_matched_merchant_lst

array(['Sunterra Market', 'TaiKo Supermarket', 'Planet Organic',
       'Panchvati Supermarket', "Lococo's", "Nature's Emporium",
       "Mike Dean's Super Food Stores"], dtype=object)

In [66]:
merchant_prod_df.head()

merchant  product_id product_name  aisle_id  \
0  L&M Markets (Hometown Grocers Co-op)         196         Soda        77   
1                       Fairway Markets         196         Soda        77   
2                             Coleman's         196         Soda        77   
3                       Saskatoon Co-op         196         Soda        77   
4                     Asian Food Centre         196         Soda        77   

   department_id        aisle department  price  
0              7  soft drinks  beverages   3.08  
1              7  soft drinks  beverages   3.08  
2              7  soft drinks  beverages   3.08  
3              7  soft drinks  beverages   3.08  
4              7  soft drinks  beverages   3.08

In [91]:
def get_merchant_discounts(merchant, recommended_set):
    merchant_prod_set = set(merchant_prod_df.loc[merchant_prod_df['merchant']==merchant]['product_name'].unique())
    common_set = merchant_prod_set.intersection(recommended_set)
    discount = round(merchant_sales_df.loc[merchant_sales_df['merchant']==merchant]['discount'].values[0], 4)
    
    total_save = round(discount * len(common_set), 4)
    return common_set, total_save

In [95]:
recommeded_merchant_dct = {}

for merchant in top_matched_merchant_lst:
    recommended_set, total_save = get_merchant_discounts(merchant, set(prod_lst))
    recommeded_merchant_dct[merchant] = {'total_save': total_save, 'recommended_set': recommended_set}
    
order_dct = {k: v for k, v in sorted(recommeded_merchant_dct.items(), key=lambda item: item[1]['total_save'], reverse=True)}

In [97]:
for merchant, recommendation in order_dct.items():
    print(merchant)
    print('Total Save $' + str(recommendation['total_save']))
    print('Recommended Items:', recommendation['recommended_set'])
    print()

Lococo's
Total Save $32.3818
Recommended Items: {'Glitter Candles', 'Organic Strained Tomatoes', 'Mini Seedless Watermelon Pack', 'Organic Tomato Paste', 'Elderflower Tonic Water', 'Fresh Cut Butternut Squash', '100% Organic Diced Tomatoes', 'Lemon Cucumber Sparkling Water', 'Cantaloupe Grapefruit Sparkling Water', 'Green Seedless Grapes', 'Red Bell Peppers'}

Mike Dean's Super Food Stores
Total Save $27.4384
Recommended Items: {'Organic Strained Tomatoes', 'Mini Seedless Watermelon Pack', 'Organic Tomato Paste', 'Elderflower Tonic Water', 'Fresh Cut Butternut Squash', '100% Organic Diced Tomatoes', 'Lemon Cucumber Sparkling Water', 'Cantaloupe Grapefruit Sparkling Water', 'Green Seedless Grapes', 'Dry Cola Soda, Lightly Sweetened', 'Red Bell Peppers'}

Nature's Emporium
Total Save $20.995
Recommended Items: {'Organic Strained Tomatoes', 'Mini Seedless Watermelon Pack', 'Organic Tomato Paste', 'Fresh Cut Butternut Squash', '100% Organic Diced Tomatoes', 'Lemon Cucumber Sparkling Water'